In [ ]:
import os
import librosa
import numpy as np
from pymongo import MongoClient
from gridfs import GridFS
from mutagen.mp3 import MP3
from mutagen.id3 import ID3

# import wave

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['Dataset']
fs = GridFS(db)


def is_valid_mp3(audio_file):
    try:
        # Check if the file extension is ".mp3"
        if not audio_file.lower().endswith('.mp3'):
            return False

        # Attempt to parse the file as an MP3
        audio = MP3(audio_file)

        # Check if the audio file has a non-zero duration
        if audio.info.length <= 0:
            return False

        # Check for corruption using mutagen's open function
        with open(audio_file, 'rb') as f:
            try:
                MP3(f)
            except Exception as e:
                print(f"Corrupted MP3 file: {audio_file}")
                return False

        return True

    except Exception as e:
        # If any other exception occurs during file processing, consider it invalid
        print(f"Error processing MP3 file: {audio_file}: {e}")
        return False

    except Exception as e:
        # If an error occurs during file processing, consider it invalid
        return False


def extract_metadata(audio_file):
    # Open the audio file
    audio = MP3(audio_file, ID3=ID3)

    # Extract metadata
    metadata = {}
    id3 = audio.tags
    if id3:
        metadata['album'] = id3.get('TALB', [''])[0]
        metadata['albumartist'] = id3.get('TPE2', [''])[0]
        metadata['artist'] = id3.get('TPE1', [''])[0]
        metadata['comment'] = id3.get('COMM', [''])[0]
        metadata['compilation'] = id3.get('TCMP', [''])[0]
        metadata['composer'] = id3.get('TCOM', [''])[0]
        metadata['discnumber'] = id3.get('TPOS', [''])[0]
        metadata['genre'] = id3.get('TCON', [''])[0]
        metadata['lyrics'] = id3.get('USLT', [''])[0]
        metadata['totaldiscs'] = id3.get('TXXX:totaldiscs', [''])[0]
        metadata['totaltracks'] = id3.get('TXXX:totaltracks', [''])[0]
        metadata['tracknumber'] = id3.get('TRCK', [''])[0]
        metadata['tracktitle'] = id3.get('TIT2', [''])[0]
        metadata['year'] = id3.get('TDRC', [''])[0]
        metadata['isrc'] = id3.get('TSRC', [''])[0]

        # Convert timestamps to strings
        metadata['year'] = str(metadata['year'])

        # Extract artwork
        if 'APIC:' in id3:
            artwork = id3['APIC:'].data
            metadata['artwork'] = artwork

    return metadata


# Function to compute features
def compute_features(audio_file):
    try:
        # Load the audio file
        y, sr = librosa.load(audio_file, res_type='kaiser_fast')

        # Compute MFCCs
        mfccs = librosa.feature.mfcc(y=y, sr=sr)

        # Compute spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

        # Compute zero-crossing rate
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

        # Compute statistics for MFCCs
        mfccs_stats = {
            "mean": np.mean(mfccs, axis=1).tolist(),
            "median": np.median(mfccs, axis=1).tolist(),
            "std": np.std(mfccs, axis=1).tolist(),
        }

        # Compute statistics for spectral centroid
        spectral_centroid_stats = {
            "mean": np.mean(spectral_centroid).tolist(),
            "median": np.median(spectral_centroid).tolist(),
            "std": np.std(spectral_centroid).tolist(),
        }

        # Compute statistics for zero-crossing rate
        zero_crossing_rate_stats = {
            "mean": np.mean(zero_crossing_rate).tolist(),
            "median": np.median(zero_crossing_rate).tolist(),
            "std": np.std(zero_crossing_rate).tolist(),
        }

        return {
            "mfccs_stats": mfccs_stats,
            "spectral_centroid_stats": spectral_centroid_stats,
            "zero_crossing_rate_stats": zero_crossing_rate_stats,
        }
    except Exception as e:
        print(f"Error processing audio file {audio_file}: {e}")
        return None



# Folder containing MP3 files
folder_path = r'F:\New folder\spotify\fma_large'

# Iterate over files in the folder
for folder_name in range(115, 155):
    folder_name = str(folder_name).zfill(3)  # Pad with zeros if needed
    folder = os.path.join(folder_path, folder_name)
    if os.path.isdir(folder):
        for filename in os.listdir(folder):
            if filename.endswith('.mp3'):
                # File path
                file_path = os.path.join(folder, filename)

                # Check if the MP3 file is valid
                if not is_valid_mp3(file_path):
                    print(f"Skipping invalid MP3 file: {filename}")
                    continue

                # Extract metadata
                metadata = extract_metadata(file_path)

                # Compute features
                features = compute_features(file_path)

                # Read audio file content
                with open(file_path, 'rb') as mp3_file:
                    mp3_content = mp3_file.read()

                # Store audio file, metadata, and features in MongoDB
                # Store audio file, metadata, and features in MongoDB
                file_id = fs.put(mp3_content, filename=filename)
                if file_id:
                    if 'features' not in locals() or features is None:
                        features = {}  # Initialize features as an empty dictionary if it doesn't exist or is None
                    features['file_id'] = file_id  # Add file ID to features
                    db.audio_metadata.insert_one({"filename": filename, "metadata": metadata, "features": features})
                    print(f"File '{filename}' processed and saved with ID: {file_id}")
                else:
                    print(f"Error processing file '{filename}'. Skipping...")

                print(f"File '{filename}' processed and saved with ID: {file_id}")

print("All files processed and saved successfully.")
